In [1]:
import matplotlib.pyplot as plt
import random as r
import pprint as pp
import numpy as np
import csv

In [2]:
# Define type of slice and its parameters
slices_type = {
    'eMBB' : {
        'bandwidth_max': 100000000, #max bandwidth which BS can give to this slice
        'client_weight': 0.3, # Portion of clients which use this type of slice
        'band_guaranteed' : 0, # based on quality of service
        'min': 4000000, 
        'max': 500000000 # how much bps can require this type of slice
      },
      'URLLC' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.2,
        'band_guaranteed' : 5000000,
        'min': 5000000,
        'max': 8000000
      },
      'MIoT' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.15,
        'band_guaranteed' : 1000000,
        'min': 1000000, 
        'max': 8000000
      },
      'mMTC' : {
        'bandwidth_max': 10000000,
        'client_weight': 0.15,
        'band_guaranteed' : 1000000,
        'min': 1000000, 
        'max': 8000000
      },
      'voice' : {
        'bandwidth_max': 1000000,
        'client_weight': 0.2,
        'band_guaranteed' : 500000,
        'min': 4000000, 
        'max': 8000000
     }
}
          
# Define base station parameters
basestation = {
    'capacity': 20000000000,   #20Gbps
    'coverage': 250,
    'ratios': { # portion of capacity dedicated to various slice type
        'URLLC': 0.05,
        'eMBB': 0.55,
        'mMTC': 0.15,
        'voice': 0.1,
        'MIoT': 0.15
    },
    'x': 300,
    'y': 300
}

# Define mobility patterns and its parameters
mobility_patterns = {
  'car': {
    'distribution': 'normal',
    'params':(0,7),
    'client_weight': 0.10
  },
  'walk': {
    'distribution': 'randint',
    'params': (-1, 1),
    'client_weight': 0.40
  },
  'stationary': {
    'distribution': 'normal',
    'params': (0, 0.1),
    'client_weight': 0.20
  },
  'publictransport': {
    'distribution': 'randint',
    'params': (-4, 4),
    'client_weight': 0.10
  },
  'slackperson':{
    'distribution': 'randint',
    'params':(0, 1),
    'client_weight': 0.20
  }
}

In [3]:
# Define obj Client
class Client:
    def __init__(self, ID, x, y, mobility_pattern, freq, slice_type, bs, slice_cap = None):
        self.ID = ID
        self.x = x
        self.y = y
        self.mobility_pattern = mobility_pattern
        self.freq = freq
        self.slice_type = slice_type
        self.bs = bs
        self.usage_rem = 0
        self.last_usage = 0
        self.connected = False
    
    def step_1(self):
        if self.usage_rem > 0:
            if self.connected:
                self.start_consume()
            else:
                self.connect() # if connection refused, usage remaining in memory
        else:
            if self.connected:
                self.disconnect()
            else:
                self.generate_usage_and_connect()
        
    def step_2(self):
        if self.connected and self.last_usage > 0:
            self.release_consume()
            if self.usage_rem == 0:
                self.disconnect()
    
    def step_3(self):
        x, y = self.move()
        self.x += x
        self.y += y    
                
    def generate_usage_and_connect(self):
        if self.slice_type is not None:
            # Generate new usage
            self.usage_rem = self.generate()
            self.connect()
            #print(f"Client_{self.ID}, coordinates : ({self.x}, {self.y}) requests {self.usage_rem} usage to BS{self.bs.ID} to slice {self.slice_type}")     

    def connect(self):
        s = self.slice_type
        
        if self.connected:
            return
        
        if self.is_avaliable():         # threshold for usage
            self.connected = True
            
            if s == 'eMBB':
                bs.embb_users += 1
            if s == 'URLLC':
                bs.urllc_users += 1
            if s == 'MIoT':
                bs.miot_users += 1
            if s == 'mMTC':
                bs.mmtc_users += 1
            if s == 'voice':
                bs.voice_users += 1
            
            #print(f"Client {self.ID}, coordinates : ({self.x}, {self.y}) connected to slice {s} @ BS{self.bs.ID}")
        else:
            self.connected = False
            #print(f"Client {self.ID}, coordinates : ({self.x}, {self.y}) connection refused to slice {s} @ BS{self.bs.ID}")
        
    def disconnect(self):
        s = self.slice_type

        if s == 'eMBB':
            bs.embb_users -= 1
        if s == 'URLLC':
            bs.urllc_users -= 1
        if s == 'MIoT':
            bs.miot_users -= 1
        if s == 'mMTC':
            bs.mmtc_users -= 1
        if s == 'voice':
            bs.voice_users -= 1
                
        self.connected = False
        self.usage_rem = 0
        
        #print(f"Client {self.ID}, coordinates: ({self.x},{self.y}) disconnected from slice {self.slice_type} @ BS{self.bs.ID} ")
    
    def start_consume(self):
        amount = min(self.usage_rem, self.get_consumable_share())    # bandwidth allocation
        self.last_usage = amount
        #print(f"Client {self.ID}, coordinates: ({self.x}, {self.y}) gets {amount} usage from @ BS{self.bs.ID} for slice {self.slice_type}")
           
    def release_consume(self):
        # release resources
        if self.last_usage > 0:
            self.usage_rem -= self.last_usage
            #print(f"Client {self.ID}, coordinates: ({self.x}, {self.y}), releases {self.last_usage} usage to BS{self.bs.ID}")
            self.last_usage = 0
        
    # update client coordinates    
    def move(self):   
        distr = self.mobility_pattern.distribution
        params = self.mobility_pattern.params
        if distr == "randint":
            delta_x = r.randint(params[0], params[1])
            delta_y = r.randint(params[0], params[1])
            return delta_x, delta_y
        if distr == "normal":
            delta_x = np.random.normal(params[0], params[1])
            delta_y = np.random.normal(params[0], params[1])
            return delta_x, delta_y 
        
    # generate client consume   
    def generate(self):
        s = self.slice_type
        cicciolina = r.randint(slices_type[s]['min'], slices_type[s]['max'])
        return cicciolina
    
    def is_avaliable(self):
        global capacity
        s = self.slice_type
        
        if s == 'eMBB':
            bandwidth_next = capacity[s] / (bs.embb_users + 1)
        if s == 'URLLC':
            bandwidth_next = capacity[s] / (bs.urllc_users + 1)
        if s == 'MIoT':
            bandwidth_next = capacity[s] / (bs.miot_users + 1)
        if s == 'mMTC':
            bandwidth_next = capacity[s] / (bs.mmtc_users + 1)
        if s == 'voice':
            bandwidth_next = capacity[s] / (bs.voice_users + 1)
       
        if bandwidth_next < slices_type[s]['band_guaranteed']:
            return False

        return True
      
    def get_consumable_share(self):
        global capacity
        s = self.slice_type

        if s == 'eMBB':
            if bs.embb_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else: 
                return int(min(capacity[s]/bs.embb_users, slices_type[s]['bandwidth_max']))
        
        if s == 'URLLC':
            if bs.urllc_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.urllc_users, slices_type[s]['bandwidth_max']))
        
        if s == 'MIoT':
            if bs.miot_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.miot_users, slices_type[s]['bandwidth_max']))
        
        if s == 'mMTC':
            if bs.mmtc_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else: 
                return int(min(capacity[s]/bs.mmtc_users, slices_type[s]['bandwidth_max']))
        
        if s == 'voice':
            if bs.voice_users <= 0:
                return int(min(capacity[s], slices_type[s]['bandwidth_max']))
            else:
                return int(min(capacity[s]/bs.voice_users, slices_type[s]['bandwidth_max']))
        

In [4]:
# Define obj Mobility 
class Mobility:
    def __init__(self, name, distribution, params):
        self.name = name
        self.distribution = distribution
        self.params = params

# Define obj BaseStation
class BaseStation:
    def __init__(self,ID, x, y, coverage, capacity, ratios):
        self.ID =ID
        self.x = x
        self.y = y
        self.coverage = coverage
        self.capacity = capacity
        self.ratios = ratios
        
        self.embb_users = 0
        self.urllc_users = 0
        self.miot_users = 0
        self.mmtc_users = 0
        self.voice_users = 0
        

In [5]:
def get_random_mobility_pattern(vals, mobility_patterns):
    i = 0
    j = r.random()

    while vals[i] < j:
        i += 1

    return mobility_patterns[i]

def get_random_slice_type(vals):
    i = 0
    j = r.random()
    s = ''

    while vals[i] < j:
        i += 1
    
    if i == 0:
       s = 'eMBB'
    if i == 1:
       s = 'URLLC' 
    if i == 2:
       s = 'MIoT' 
    if i == 3:
       s = 'mMTC' 
    if i == 4:
       s = 'voice' 
    
    return s

def inRange(client, bs):
    x_client = client.x
    y_client = client.y
    x_bs = bs.x
    y_bs = bs.y
    if (abs(x_client - x_bs) < bs.coverage) and (abs(y_client - y_bs) < bs.coverage):
        return True
        print('true')
    else:
        return False
        print('false')

collected, slice_weights = 0, []
for key, value in slices_type.items():
    collected += value['client_weight']
    slice_weights.append(collected)

collected, mb_weights = 0, []
for key, value in mobility_patterns.items():
    collected += value['client_weight']
    mb_weights.append(collected)
    
m_patterns = []
for name, mb in mobility_patterns.items():
    mobility_pattern = Mobility(name, mb['distribution'], mb['params'])
    m_patterns.append(mobility_pattern)   


In [6]:
# Define class Request
class Request:
    def __init__(self, id_client, type_slice, traffic_request, traffic_left):
        self.id_client = id_client
        self.type_slice = type_slice
        self.traffic_request = traffic_request
        self.traffic_left = traffic_left
        
#Define class csv
class CsvData:
    def __init__(self, step, r_embb, g_embb, connect_embb, r_urllc, g_urllc, connect_urllc, r_miot, g_miot, connect_miot, r_mmtc, g_mmtc, connect_mmtc, r_voice, g_voice, connect_voice):
        self.step = step
        self.r_embb = r_embb
        self.g_embb = g_embb
        self.connect_embb = connect_embb
        self.r_urllc = r_urllc
        self.g_urllc = g_urllc
        self.connect_urllc = connect_urllc
        self.r_miot = r_miot
        self.g_miot = g_miot
        self.connect_miot = connect_miot
        self.r_mmtc = r_mmtc
        self.g_mmtc = g_mmtc
        self.connect_mmtc = connect_mmtc
        self.r_voice = r_voice
        self.g_voice = g_voice
        self.connect_voice = connect_voice
        

In [7]:
# Round 0: create network area, place base station and clients
num_clients = 2000
simulation_time = 1000
x_area = 600
y_area = 600

bs = BaseStation(0, basestation.get('x'), basestation.get('y'), basestation.get('coverage'), basestation.get('capacity'), basestation.get('ratios'))

clients = []
for i in range(num_clients):
    x = r.randint(0, x_area)
    y = r.randint(0, y_area)
    freq = r.random()
    
    mobility_pattern = get_random_mobility_pattern(mb_weights, m_patterns)
    slice_type = get_random_slice_type(slice_weights) 
    
    c = Client(i, x, y, mobility_pattern, freq, slice_type, bs)
    clients.append(c)


In [8]:
active_clients = []
request_list = [None] * len(clients) # create fixed size list to manage the request
csv_final = []

for n in range(simulation_time):
    
    # take only client in range
    clients_inRange = []
    for item in clients:
        if inRange(item, bs):
            clients_inRange.append(item)
    
    # manage number active clients
    active = []
    new_connection = r.randint(0, len(clients_inRange))
    active = r.sample(clients_inRange, new_connection)
    
    for item in active:
        if item.connected == False:
            active_clients.append(item)
    
    capacity = { 'eMBB' : 11000000000,
                 'URLLC' :1000000000,
                 'MIoT' : 3000000000,
                 'mMTC' : 3000000000,
                 'voice' : 2000000000
               }
    
    print('STEP', n)
    
    for item in active_clients:   
        # step 1 and create request
        item.step_1()
        
        req = Request(item.ID, item.slice_type,item.usage_rem, item.usage_rem)
        request_list[item.ID] = req
  
    for item in active_clients:
        # step 2 and update request (delete non active user, and update traffic)
        item.step_2()
        
        request_list[item.ID].traffic_left = item.usage_rem
    
    user_embb, user_urllc, user_miot, user_mmtc, user_voice = 0, 0, 0, 0, 0
    req_embb, req_urllc, req_miot, req_mmtc, req_voice = 0, 0, 0, 0, 0
    giv_embb, giv_urllc, giv_miot, giv_mmtc, giv_voice = 0, 0, 0, 0, 0
    for item in request_list:
     if item is not None:
        if item.type_slice == 'eMBB':
            if item.traffic_request - item.traffic_left != 0:
                req_embb += item.traffic_request
                giv_embb += item.traffic_request - item.traffic_left 
                user_embb += 1
            else:
                req_embb += 0
                giv_embb += 0
                
        if item.type_slice == 'URLLC' :
            if item.traffic_request - item.traffic_left != 0:
                req_urllc += item.traffic_request
                giv_urllc += item.traffic_request - item.traffic_left
                user_urllc += 1
            else:
                req_urllc += 0
                giv_urllc += 0
 
        if item.type_slice == 'MIoT':
            if item.traffic_request - item.traffic_left != 0:
                req_miot += item.traffic_request
                giv_miot += item.traffic_request - item.traffic_left
                user_miot += 1
            else:
                req_miot += 0
                giv_miot += 0
 
        if item.type_slice == 'mMTC':
            if item.traffic_request - item.traffic_left != 0:
                req_mmtc += item.traffic_request
                giv_mmtc += item.traffic_request - item.traffic_left
                user_mmtc += 1
            else:
                req_mmtc += 0
                giv_mmtc += 0

        if item.type_slice == 'voice':
            if item.traffic_request - item.traffic_left != 0:
                req_voice += item.traffic_request
                giv_voice += item.traffic_request - item.traffic_left
                user_voice += 1
            else:
                req_voice += 0
                giv_voice += 0
    
    csv_data = CsvData(n, req_embb, giv_embb, user_embb, req_urllc, giv_urllc, user_urllc, req_miot, giv_miot, user_miot, req_mmtc, giv_mmtc, user_mmtc, req_voice, giv_voice, user_voice)  
    csv_final.append(csv_data)

    for item in clients:
        # step 3 move all clients
        item.step_3()
    
    tmp = []
    for item in active_clients:    
        if not inRange(item, bs):
            item.disconnect()
        if item.connected == True:
            request_list[item.ID] = None
            tmp.append(item)
    
    active_clients = tmp
            

STEP 0
395
STEP 1
386
STEP 2
366
STEP 3
357
STEP 4
358
STEP 5
354
STEP 6
344
STEP 7
335
STEP 8
375
STEP 9
359
STEP 10
350
STEP 11
335
STEP 12
356
STEP 13
317
STEP 14
261
STEP 15
217
STEP 16
209
STEP 17
220
STEP 18
298
STEP 19
310
STEP 20
276
STEP 21
242
STEP 22
276
STEP 23
293
STEP 24
334
STEP 25
326
STEP 26
320
STEP 27
313
STEP 28
330
STEP 29
293
STEP 30
367
STEP 31
338
STEP 32
331
STEP 33
368
STEP 34
351
STEP 35
319
STEP 36
351
STEP 37
380
STEP 38
369
STEP 39
348
STEP 40
320
STEP 41
297
STEP 42
366
STEP 43
342
STEP 44
332
STEP 45
323
STEP 46
299
STEP 47
348
STEP 48
334
STEP 49
312
STEP 50
296
STEP 51
262
STEP 52
234
STEP 53
233
STEP 54
324
STEP 55
294
STEP 56
348
STEP 57
348
STEP 58
357
STEP 59
372
STEP 60
373
STEP 61
357
STEP 62
332
STEP 63
316
STEP 64
359
STEP 65
353
STEP 66
333
STEP 67
308
STEP 68
328
STEP 69
347
STEP 70
313
STEP 71
311
STEP 72
280
STEP 73
310
STEP 74
338
STEP 75
319
STEP 76
351
STEP 77
357
STEP 78
336
STEP 79
356
STEP 80
318
STEP 81
290
STEP 82
352
STEP 83
333
ST

STEP 646
181
STEP 647
175
STEP 648
132
STEP 649
73
STEP 650
204
STEP 651
172
STEP 652
138
STEP 653
213
STEP 654
191
STEP 655
206
STEP 656
193
STEP 657
172
STEP 658
146
STEP 659
95
STEP 660
75
STEP 661
166
STEP 662
176
STEP 663
204
STEP 664
212
STEP 665
219
STEP 666
182
STEP 667
183
STEP 668
142
STEP 669
116
STEP 670
124
STEP 671
99
STEP 672
191
STEP 673
206
STEP 674
160
STEP 675
151
STEP 676
169
STEP 677
117
STEP 678
172
STEP 679
121
STEP 680
104
STEP 681
61
STEP 682
18
STEP 683
-18
STEP 684
-45
STEP 685
-65
STEP 686
-70
STEP 687
-75
STEP 688
-76
STEP 689
-76
STEP 690
-76
STEP 691
-76
STEP 692
-76
STEP 693
-77
STEP 694
-79
STEP 695
-79
STEP 696
-79
STEP 697
-79
STEP 698
-79
STEP 699
-79
STEP 700
-80
STEP 701
-80
STEP 702
-81
STEP 703
-81
STEP 704
-81
STEP 705
-82
STEP 706
-82
STEP 707
-82
STEP 708
-82
STEP 709
-82
STEP 710
-82
STEP 711
-82
STEP 712
-83
STEP 713
-83
STEP 714
-83
STEP 715
-84
STEP 716
-85
STEP 717
-86
STEP 718
-88
STEP 719
-88
STEP 720
-88
STEP 721
-89
STEP 722
-90
STEP 

In [9]:
with open('input.csv', mode='w') as csv_file:
    fieldnames = ['embb_rate', 'embb_users', 'embb_req', 'urllc_rate', 'urllc_users', 'urllc_req', 'miot_rate', 'miot_users', 'miot_req', 'mmtc_rate', 'mmtc_users', 'mmtc_req', 'voice_rate', 'voice_users', 'voice_req']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    
    writer.writeheader()
    for item in csv_final:
        writer.writerow({'embb_rate': item.g_embb, 'embb_users': item.connect_embb, 'embb_req': item.r_embb, 'urllc_rate': item.g_urllc, 'urllc_users': item.connect_urllc, 'urllc_req': item.r_urllc, 'miot_rate': item.g_miot, 'miot_users': item.connect_miot, 'miot_req': item.r_miot, 'mmtc_rate': item.g_mmtc, 'mmtc_users': item.connect_mmtc, 'mmtc_req': item.r_mmtc, 'voice_rate': item.g_voice, 'voice_users': item.connect_voice, 'voice_req': item.r_voice})
